## Calculate the TSN intrinsic value

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
from math import sqrt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

In [13]:
#load data
path_data = '../input/TSN_Q_exp.csv'
data = pd.read_csv(path_data, sep=';', parse_dates=[2])
# filter data
report = 'Q'
start_date = pd.Timestamp('2013-01-01')
data_filt = data.loc[(data['report'] == report) & (data['date'] > start_date), :]

data_filt.reset_index(drop=True, inplace=True)

In [14]:
for column in data_filt.columns[3:]:
    data_filt.loc[:, column] = data_filt.loc[:, column].astype('float')

In [15]:
data_filt.head()

,ticker,report,date,rev,cost_sale,r_d,sale_gen_adm,inpairm,other_charg,op_income,...,short_debt,acc_pay,other_curr_liab,tot_curr_liab,tot_long_debt,other_liab,tot_equity,tot_liab,depr_amort,stock_b_comp
0,TSN,Q,2013-04-01,8419,8011,0,234,0,0,174,...,512,1265,955,2732,1904.0,541,6114,5675,129,9.00
1,TSN,Q,2013-07-01,8731,8049,0,263,0,0,419,...,508,1309,1121,2938,1899.0,551,6247,5855,128,9.00
2,TSN,Q,2013-10-01,8822,8091,0,249,0,0,482,...,513,1359,1138,3010,1895.0,560,6233,5944,132,9.00
3,TSN,Q,2014-01-01,8761,8076,0,273,0,0,412,...,52,1477,1077,2606,1890.0,582,6316,5528,127,12.75
4,TSN,Q,2014-04-01,9032,8381,0,290,0,0,361,...,52,1429,1024,2505,1888.0,585,6464,5422,127,12.75


### Calculate EBITDA, EBIT, NOPAT, CAPEX etc on hisorical data

In [28]:
def calculate_missing_columns(df):
    df.loc[:, 'ebitda'] = df['rev'] - df['cost_sale'] - df['sale_gen_adm'] - df['r_d'] - df['other_charg']
    df.loc[:, 'ebit'] = df['ebitda'] - df['depr_amort']
    df.loc[:, 'nopat'] = df['ebit'] - df['inc_tax_exp']
    df.loc[:, 'other_curr_ass_lesscash'] = df['acc_rec'] + df['invent'] + df['other_curr_ass']
    df.loc[:, 'tot_curr_liab_lessdebt'] = df['tot_curr_liab'] - df['short_debt']
    df.loc[:, 'nwc'] = df['other_curr_ass_lesscash'] - df['tot_curr_liab_lessdebt']
    # capex calculation
    capex = []
    for index in range(len(df)):
        if index > 0:
            ppe_change = df['ppe_net'].iloc[index] - df['ppe_net'].iloc[index-1]
            depr_amort = df['depr_amort'].iloc[index]
            capex.append(ppe_change + depr_amort)
        else:
            capex.append(np.nan)
    df.loc[:, 'capex'] = capex

    return df

In [29]:
data_filt = calculate_missing_columns(data_filt)
data_filt.head()

,ticker,report,date,rev,cost_sale,r_d,sale_gen_adm,inpairm,other_charg,op_income,...,tot_liab,depr_amort,stock_b_comp,ebitda,ebit,nopat,other_curr_ass_lesscash,tot_curr_liab_lessdebt,nwc,capex
0,TSN,Q,2013-04-01,8419,8011,0,234,0,0,174,...,5675,129,9.00,174,45,-8,4537,2220,2317,NaN
1,TSN,Q,2013-07-01,8731,8049,0,263,0,0,419,...,5855,128,9.00,419,291,155,4584,2430,2154,168.0
2,TSN,Q,2013-10-01,8822,8091,0,249,0,0,482,...,5944,132,9.00,482,350,226,4459,2497,1962,143.0
3,TSN,Q,2014-01-01,8761,8076,0,273,0,0,412,...,5528,127,12.75,412,285,154,4405,2554,1851,146.0
4,TSN,Q,2014-04-01,9032,8381,0,290,0,0,361,...,5422,127,12.75,361,234,103,4746,2453,2293,160.0


### Revenue Forecasts